In [47]:
import requests
fsave='./HARP_TO_NOAA.txt'
def fetch_newest_harpnum_to_noaa(filename=fsave):
    '''
    This function downloads newest harp to noaa mapping from jsoc
    just call the function.
    '''
    url = 'http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt'
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)

##fetch_newest_harpnum_to_noaa()

In [48]:
def harp_from_noaa(NOAANUM,noaalist):
    '''
    This returns HARP number for provided NOAA region from content file
    '''
    # Read content of HARPS TO NOAA in content
    with open(noaalist) as f:
        content = [line.rstrip() for line in f]
    ins = [index for index, string in enumerate(content) if NOAANUM in string]
    harpnum = [ content[x].split(' ')[0] for x in ins ]
    if len(harpnum) != 1:
        raise Exception("Be careful! Your region is over multiple HARPs or it was not found! HARPS: {}".format(harpnum))
    return harpnum[0]

In [49]:
HARPNUM=harp_from_noaa("11950",fsave)
MY_MAIL='lazar.zivadinovic.994@gmail.com'
print(HARPNUM)

3604


In [42]:
#Lets find primary keys and data sets for our set
import drms
c = drms.Client()
si = c.info('hmi.sharp_cea_720s')
print(si.segments)
# ['magnetogram' 'bitmap' 'Dopplergram' 'continuum' 'Bp' 'Bt' 'Br' 'Bp_err'
# 'Bt_err' 'Br_err' 'conf_disambig']

print(c.pkeys('hmi.sharp_cea_720s'))
# ['HARPNUM', 'T_REC']

               type       units protocol     dims  \
name                                                
magnetogram     int       Gauss     fits  VARxVAR   
bitmap         char  Enumerated     fits  VARxVAR   
Dopplergram     int         m/s     fits  VARxVAR   
continuum       int        DN/s     fits  VARxVAR   
Bp              int       Gauss     fits  VARxVAR   
Bt              int       Gauss     fits  VARxVAR   
Br              int       Gauss     fits  VARxVAR   
Bp_err          int       Gauss     fits  VARxVAR   
Bt_err          int       Gauss     fits  VARxVAR   
Br_err          int       Gauss     fits  VARxVAR   
conf_disambig  char        none     fits  VARxVAR   

                                                      note  
name                                                        
magnetogram    Line-of-sight magnetogram in CEA projection  
bitmap               Mask for the patch in CEA coordinates  
Dopplergram                  Dopplergram in CEA projection  
conti

In [43]:
import os
# Lets make jsoc query 
from sunpy.net import jsoc
from sunpy.net import attrs as a
from sunpy.time import parse_time
#initialize client
client = jsoc.JSOCClient()

data_root='./data'
download_path=os.path.join(data_root,str(HARPNUM))

###### THIS IS THE REAL QUERY, LETS MAKE SOMETHING SMALLER FOR TESTING ########
# Create query
resjsoc = client.search(a.jsoc.PrimeKey('HARPNUM', HARPNUM),
                        a.jsoc.Series('hmi.sharp_cea_720s'),
                        a.jsoc.Segment('Bp') & a.jsoc.Segment('Bt') & 
                        a.jsoc.Segment('Br') & a.jsoc.Segment('continuum') &
                        a.jsoc.Segment('bitmap'),
                        a.jsoc.Notify(MY_MAIL))

#Dont try to fix max_conns, jsoc will block you (max conns from ip)
#https://github.com/sunpy/sunpy/pull/3822
#result = client.fetch(resjsoc, path=download_path, progress=True, wait=True)
#Use result as you like! Its list of filenames!